In [1]:
import struct
import numpy as np
from PIL import Image
from pathlib import Path
import os
import shutil
import traceback
import sys
import geopandas as gpd
from tqdm import tqdm
import subprocess
import re
from pyproj import transform, Proj
from shapely import geometry, affinity
from rasterio.control import GroundControlPoint
from rasterio.transform import from_gcps
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from osgeo import gdal, osr; gdal.UseExceptions()
import pickle
from IPython.display import clear_output
import warnings; warnings.filterwarnings("ignore")


In [2]:
distance = 25
humidity = 90
emissivity = 0.95
reflection = 0

In [3]:
# directory structure prparation
rjpg_dir = 'data/input/rjpg'
tiff_dir = 'data/output/tiff'
tmp_dir = 'data/tmp'
if os.path.exists(tmp_dir):
    shutil.rmtree(tmp_dir)
os.mkdir(tmp_dir)

# R-JPG to TIFF

In [4]:
distance = round(distance)
humidity = round(humidity)
emissivity = round(emissivity,2)
reflection = round(reflection)

assert os.path.exists(rjpg_dir)
if os.path.exists(tiff_dir):
    shutil.rmtree(tiff_dir)
os.makedirs(tiff_dir)
# get list of .jpg files from input directory
file_list = list(filter(lambda x:x.endswith((".jpg", ".JPG")), os.listdir(rjpg_dir)))
# with open("data/input/files.txt") as f:
#     file_list = f.read().strip().splitlines()
# iterate over files
for i, file_name in enumerate(file_list):
    # get temporary raw byte file by DJI Thermal SDK
    #clear cell output
    clear_output(wait=True)
    print(f"File {i+1}/{len(file_list)}")
    os.system(  f"./dji_irp -a measure \
                --measurefmt float32 \
                --distance {distance} \
                --humidity {humidity} \
                --emissivity {emissivity} \
                --reflection {reflection} \
                -s {rjpg_dir}/{file_name} -o {tmp_dir}/bytes.raw")
    # get image size
    img = Image.open(f"{rjpg_dir}/{file_name}")
    # decode temporary byte file to tiff
    arr = np.zeros(img.size[0]*img.size[1])
    with open(f"{tmp_dir}/bytes.raw", "rb") as f:
        data = f.read()
        format = '{:d}f'.format(len(data)//4)
        arr=np.array(struct.unpack(format, data))
    arr = arr.reshape(img.size[1],img.size[0])
    im = Image.fromarray(arr)
    im.save(f"{tiff_dir}/{file_name}.tiff")
    # copy exif data from original file to new tiff
    os.system(f"exiftool -tagsfromfile {rjpg_dir}/{file_name} {tiff_dir}/{file_name}.tiff -overwrite_original_in_place")

File 453/453
DIRP API version number : 0x13
DIRP API magic version  : d4c7dea
R-JPEG file path : data/input/rjpg/DJI_20220324155129_0453_T.JPG
R-JPEG version information
    R-JPEG version : 0x1
    header version : 0x103
 curve LUT version : 0x1
R-JPEG resolution size
      image  width : 640
      image height : 512
Change distance from 5 to 25
Change humidity from 70 to 90
Change emissivity from 1 to 0.95
Change reflection from 23 to 0
Measurement: get params range:
distance: [1,25]
humidity: [20,100]
emissivity: [0.1,1]
reflection: [-40,500]
Run action 1
Save image file as : data/tmp/bytes.raw
Test done with return code 0
    1 image files updated


# Cleanup

In [5]:
#clear temporary files
shutil.rmtree(tmp_dir)